<a href="https://colab.research.google.com/github/shige-compit/silicon-notebooks/blob/main/xls-workshop-openlane-ja.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# XLSとOpenLaneを使ったコードからの半導体設計

```
Copyright 2021 Google LLC.
SPDX-License-Identifier: Apache-2.0
```
このノートブックでは
- 高位合成ツールキットの[XLS](https://google.github.io/xls/)での設計
- [OpenLane](https://github.com/The-OpenROAD-Project/OpenLane/)による、RTLからGDSの生成
- オープンソースの[SKY130](https://github.com/google/skywater-pdk/) PDK向けのチップ設計

を取り扱います。ソフトウェア開発に近い形でのハードウェア設計を体験してみましょう。

In [ ]:
%pip install -q https://github.com/conda-incubator/condacolab/archive/28521d7c5c494dd6377bb072d97592e30c44609c.tar.gz
#@title conda環境のインストール {display-mode: "form"}
#@markdown - ▷ ボタンをクリックすると、conda-edaのセットアップが開始されます。
#@markdown - Click the ▷ button to setup the digital design environment based on [conda-eda](https://github.com/hdl/conda-eda).

openlane_version = 'latest' #@param {type:"string"}
open_pdks_version = 'latest' #@param {type:"string"}
xls_version = 'latest' #@param {type:"string"}

if openlane_version == 'latest':
  openlane_version = ''
if open_pdks_version == 'latest':
  open_pdks_version = ''
if xls_version == 'latest':
  xls_version = ''

import os
import pathlib
import sys

!curl -Ls https://micro.mamba.pm/api/micromamba/linux-64/latest | tar -xj bin/micromamba
conda_prefix_path = pathlib.Path('conda-env')
site_package_path = conda_prefix_path / 'lib/python3.7/site-packages'
sys.path.append(str(site_package_path.resolve()))
CONDA_PREFIX = str(conda_prefix_path.resolve())
PATH = os.environ['PATH']
LD_LIBRARY_PATH = os.environ.get('LD_LIBRARY_PATH', '')
%env CONDA_PREFIX={CONDA_PREFIX}
%env PATH={CONDA_PREFIX}/bin:{PATH}
%env LD_LIBRARY_PATH={CONDA_PREFIX}/lib:{LD_LIBRARY_PATH}
!bin/micromamba create --yes --prefix $CONDA_PREFIX
!echo 'python ==3.7*' >> {CONDA_PREFIX}/conda-meta/pinned
!CI=0 bin/micromamba install --quiet --yes --prefix $CONDA_PREFIX \
                     --channel litex-hub \
                     --channel main \
                     openlane={openlane_version} \
                     open_pdks.sky130a={open_pdks_version} \
                     xls={xls_version}
!curl -L -O https://patch-diff.githubusercontent.com/raw/The-OpenROAD-Project/OpenLane/pull/1503.patch
!patch -p1 -d conda-env/share/openlane < 1503.patch
!curl -L -O https://github.com/google/xls/archive/refs/heads/main.tar.gz
!tar --strip-components=1 -xf main.tar.gz xls-main/xls/dslx/stdlib/ xls-main/xls/modules/
def2gds_mag = '''gds read $::env(CONDA_PREFIX)/share/pdk/sky130A/libs.ref/sky130_fd_sc_hd/gds/sky130_fd_sc_hd.gds
lef read $::env(CONDA_PREFIX)/share/pdk/sky130A/libs.ref/sky130_fd_sc_hd/techlef/sky130_fd_sc_hd__nom.tlef
lef read $::env(CONDA_PREFIX)/share/pdk/sky130A/libs.ref/sky130_fd_sc_hd/lef/sky130_fd_sc_hd.lef
def read $::env(IN_DEF)
gds write $::env(IN_DEF).gds'''
with open('def2gds.mag', 'w') as f:
  f.write(def2gds_mag)
!git clone https://github.com/mbalestrini/GDS2glTF.git
!python -m pip install -r GDS2glTF/requirements.txt
!git clone https://github.com/proppy/gds_viewer.git
import jinja2
gds_viewer = jinja2.Environment(loader=jinja2.FileSystemLoader('gds_viewer')).get_template('viewer.html')

## 高位合成(HLS)を使った設計

[XLS](https://google.github.io/xls/)は高位合成(High Level Synthesis: HLS)のツールチェインを提供します。XLSを用いることで、ソフトウェア開発に近い方法でハードウェアの設計が可能です。

[DSLX](https://google.github.io/xls/dslx_reference/) はハードウェア設計向けのデータフロー志向・関数型のドメイン特化言語(Domain Specific Language: DSL)です。DSLXで記述した高レベルな機能の設計から、具体的なハードウェアデザインを生成できます。

![img](https://google.github.io/xls/images/xls_stack_diagram.png)

### DSLX

>DSLX mimics Rust, while being an immutable expression-based dataflow DSL with hardware-oriented features; e.g. arbitrary bitwidths, entirely fixed size objects, fully analyzeable call graph, etc. To avoid arbitrary new syntax/semantics choices, the DSL mimics Rust where it is reasonably possible; for example, integer conversions all follow the same semantics as Rust.

>Note: There are some unnecessary differences today from Rust syntax due to early experimentation, but they are quickly being removed to converge on Rust syntax.

https://google.github.io/xls/dslx_reference/

以下の例を通して次のようなDSLXの機能を紹介します。
- 基本的な言語仕様と[文法](https://google.github.io/xls/dslx_reference/#expressions)
- [ユニットテスト](https://google.github.io/xls/dslx_reference/#unit-tests)
- [パラメトリックな関数](https://google.github.io/xls/dslx_reference/#parametric-functions)の定義
- [標準ライブラリ](https://google.github.io/xls/dslx_std/)とモジュールの[インポート](https://google.github.io/xls/dslx_reference/#imports).


In [ ]:
%%bash -c 'cat > user_module.x; interpreter_main user_module.x'

// 最初の行では、"adder1" という名前の関数 (fn) を宣言している (Functions: https://google.github.io/xls/dslx_reference/#functions)
// この関数は "a" と "b" という 2 つの 1 ビット符号なし整数 (u1) を受け取り、2 ビット符号なし整数 (u2) を返す
// この例では、"-> u2"の部分を削除することで、戻り値のない関数を定義することもできる

// 2行目は "n" という名前の2ビット符号なし整数（u2）を定義し、"a" と "b" の和を代入している
// "let" ステートメントで新しい変数を導入する
// "as" でu1型変数 "a" をu2型変数に変換（Type cast：https://google.github.io/xls/dslx_reference/#type-casting）

// 3行目は戻り値(関数は最後に計算した式の結果を戻り値として返す)
fn adder1(a: u1, b: u1) -> u2 {
  let n: u2 = a as u2 + b as u2;
  n
}


// "adder1"の関数を8ビット入力8ビット出力に変換するラッパー関数
// ビットの抽出("io_in[0:1]")についてはDSLX referenceの "Bit slice expressions" 項を参照（Bit slice expressions：https://google.github.io/xls/dslx_reference/#bit-slice-expressions）
fn user_module(io_in: u8) -> u8 {
  adder1(io_in[0:1], io_in[4:5]) as u8
}


// 関数が意図したとおりに動作するかをテストするtest function
// DSLX referenceの "assert_eq, assert_lt" 項を参照（assert_eq, assert_lt: https://google.github.io/xls/dslx_reference/#assert_eq-assert_lt）
// "let _ = ..."部分のアンダースコア(_)は、その関数の戻り値を無視することを意味する
// バイナリ値("0b0001_0001")のアンダースコア(_)は可読性のために挿入されているだけなので無視してもOK
#[test]
fn test() {
  let _ = assert_eq(adder1(u1:0b1, u1:0b1), u2:0b10);
  let _ = assert_eq(user_module(u8:0b0001_0001), u8:0b000000_10);
  _
}

## ハードウェアIRへの変換
DSLXのコードからハードウェア回路合成により適した形式
[XLS IR](https://google.github.io/xls/ir_semantics/)に変換しましょう。
XLS IRは回路合成に特化した純粋なデータフロー志向の中間表現(Intermediate Respresentation: IR)です。

In [ ]:
!ir_converter_main --top=user_module user_module.x > user_module.ir
!opt_main user_module.ir > user_module_opt.ir
!cat user_module_opt.ir

## RTLの生成

XLS codegenを使うことで、XLS IRから回路合成とシミュレーションに使用する
(System) Verilog [RTL](https://en.wikipedia.org/wiki/Register-transfer_level)を生成できます。

[Verilog](https://en.wikipedia.org/wiki/Verilog)は回路設計で広く用いられているので、XLSで生成されたVerilogコードは、各種の設計フローや他の設計と統合することができます。

In [ ]:
!codegen_main --use_system_verilog=false --module_name=user_module --generator=combinational user_module_opt.ir > user_module.v
!cat user_module.v

## OpenLaneフローの実行

[OpenLane](https://openlane.readthedocs.io/en/latest/)
は[RTL](https://en.wikipedia.org/wiki/Register-transfer_level)
から
[GDSII](https://en.wikipedia.org/wiki/GDSII)
を生成する自動化されたフローです。

このフローは
[OpenROAD](https://theopenroadproject.org/),
[Yosys](https://yosyshq.net/yosys/), [Magic](http://www.opencircuitdesign.com/magic/), [Netgen](http://opencircuitdesign.com/netgen/)
といったコンポーネントと
[open source PDKs](https://github.com/google/open-source-pdks)向けのデザイン探索や最適化のためのカスタムスクリプトからなります。

フローの概要については下記の図を参考にしてください。

![img](https://openlane.readthedocs.io/en/latest/_images/flow_v1.png)

#### OpenLaneの設定

[ドキュメント](https://openlane.readthedocs.io/en/latest/reference/configuration.html)

In [ ]:
%%writefile config.json
{
    "DESIGN_NAME": "user_module",
    "VERILOG_FILES": "dir::user_module.v",
    "CLOCK_TREE_SYNTH": false,
    "CLOCK_PERIOD": 10,
    "CLOCK_PORT": "clk",
    "CLOCK_NET": "ref::$CLOCK_PORT",
    "FP_SIZING": "absolute",
    "DIE_AREA": "0 0 50 50",
    "PL_TARGET_DENSITY": 0.30,
    "FP_PIN_ORDER_CFG": "dir::pin_order.cfg"
}

In [ ]:
%%writefile pin_order.cfg
#BUS_SORT

#W
io_in.*

#E
out.*

### 回路合成

- 入力: [RTL](https://en.wikipedia.org/wiki/Register-transfer_level) (Verilog)
- 出力: 素子情報付きの[ネットリスト](https://en.wikipedia.org/wiki/Netlist) (Verilog)
- メトリック: セルの数と [タイミング収束](https://en.wikipedia.org/wiki/Timing_closure) の推定値

[ドキュメント](https://openlane.readthedocs.io/en/latest/usage/hardening_macros.html#synthesis)

In [ ]:
%env PDK=sky130A
!flow.tcl -design . -to synthesis

In [ ]:
#@title ネットリストのプレビュー {display-mode: "form"}

import graphviz
import pathlib

dots = sorted(pathlib.Path('runs').glob('*/tmp/synthesis/post_techmap.dot'))
print(dots)
dot = graphviz.Source.from_file(dots[-1])
dot.engine = 'dot'
dot

### フロアプラン

- 入力: 素子情報付きの[ネットリスト](https://en.wikipedia.org/wiki/Netlist) (Verilog)
- 出力: 電源供給配線網(Power Delivery Network: PDN)とI/Oピンのついたダイ上のレイアウト([DEF](https://en.wikipedia.org/wiki/Design_Exchange_Format))
- メトリック: コアの面積

[ドキュメント](https://openlane.readthedocs.io/en/latest/usage/hardening_macros.html#floorplan)

In [ ]:
%env PDK=sky130A
!flow.tcl -design . -to floorplan

In [ ]:
#@title プレビュー {display-mode: "form"}

import pathlib

STEP='floorplan'
in_def = sorted(pathlib.Path('runs').glob(f'*/results/{STEP}/*.def'))[-1].resolve()
!IN_DEF={in_def} magic -dnull -noconsole -rcfile {CONDA_PREFIX}/share/pdk/sky130A/libs.tech/magic/sky130A.magicrc < def2gds.mag
!python3 GDS2glTF/gds2gltf.py {in_def}.gds | tee gds2gltf.log
import IPython.display
IPython.display.clear_output(wait=True)
with open(f'{in_def}.gds.gltf') as f:
  gltf_data=f.read()
  output = gds_viewer.render(gltf_data=gltf_data)
IPython.display.HTML(output)

### 配置

- 入力: 素子情報付きの[ネットリスト](https://en.wikipedia.org/wiki/Netlist) (Verilog),
PDNとI/Oピンつきのダイの物理レイアウト([DEF](https://en.wikipedia.org/wiki/Design_Exchange_Format))
- 出力: コンポーネントのセルが配置された物理レイアウト ([DEF](https://en.wikipedia.org/wiki/Design_Exchange_Format))
- メトリック: コア面積, セル密度, [タイミング収束](https://en.wikipedia.org/wiki/Timing_closure)の推定値

[ドキュメント](https://openlane.readthedocs.io/en/latest/usage/hardening_macros.html#placement)

In [ ]:
%env PDK=sky130A
!flow.tcl -design . -to placement

In [ ]:
#@title プレビュー {display-mode: "form"}

import pathlib

STEP='placement'
in_def = sorted(pathlib.Path('runs').glob(f'*/results/{STEP}/*.def'))[-1].resolve()
!IN_DEF={in_def} magic -dnull -noconsole -rcfile {CONDA_PREFIX}/share/pdk/sky130A/libs.tech/magic/sky130A.magicrc < def2gds.mag
!python3 GDS2glTF/gds2gltf.py {in_def}.gds | tee gds2gltf.log
import IPython.display
IPython.display.clear_output(wait=True)
with open(f'{in_def}.gds.gltf') as f:
  gltf_data=f.read()
  output = gds_viewer.render(gltf_data=gltf_data)
IPython.display.HTML(output)

### 配線

- 入力: 素子情報付きの[ネットリスト](https://en.wikipedia.org/wiki/Netlist) (Verilog),
コンポーネントのセルが配置された物理レイアウト([DEF](https://en.wikipedia.org/wiki/Design_Exchange_Format))

- 出力: コンポーネントのセルへの配線が完了した物理レイアウト ([DEF](https://en.wikipedia.org/wiki/Design_Exchange_Format))
- メトリック: ルートの衝突, [タイミング収束](https://en.wikipedia.org/wiki/Timing_closure)の予測値

[ドキュメント](https://openlane.readthedocs.io/en/latest/usage/hardening_macros.html#routing)

In [ ]:
%env PDK=sky130A
!flow.tcl -design . -to routing

In [ ]:
#@title プレビュー {display-mode: "form"}

import pathlib

STEP='routing'
in_def = sorted(pathlib.Path('runs').glob(f'*/results/{STEP}/*.def'))[-1].resolve()
!IN_DEF={in_def} magic -dnull -noconsole -rcfile {CONDA_PREFIX}/share/pdk/sky130A/libs.tech/magic/sky130A.magicrc < def2gds.mag
!python3 GDS2glTF/gds2gltf.py {in_def}.gds | tee gds2gltf.log
import IPython.display
IPython.display.clear_output(wait=True)
with open(f'{in_def}.gds.gltf') as f:
  gltf_data=f.read()
  output = gds_viewer.render(gltf_data=gltf_data)
IPython.display.HTML(output)

### サインオフ

- 入力: コンポーネントのセルへの配線が完了した物理レイアウト([DEF](https://en.wikipedia.org/wiki/Design_Exchange_Format))
- 出力: ファウンドリの[DRCルール](https://en.wikipedia.org/wiki/Design_rule_checking), 検証済みで製造に出せる物理レイアウト ([GDSII](https://en.wikipedia.org/wiki/GDSII))
- メトリック: DRCエラー, [寄生データ](https://en.wikipedia.org/wiki/Standard_Parasitic_Exchange_Format), [タイミング収束](https://en.wikipedia.org/wiki/Timing_closure)の予測値

[ドキュメント](https://openlane.readthedocs.io/en/latest/usage/hardening_macros.html#final-reports-and-checks)

In [ ]:
%env PDK=sky130A
!flow.tcl -design .

In [ ]:
#@title プレビュー {display-mode: "form"}

import pathlib

gds = sorted(pathlib.Path('runs').glob(f'*/results/final/gds/*.gds'))[-1].resolve()
!python3 GDS2glTF/gds2gltf.py {gds} | tee gds2gltf.log
import IPython.display
IPython.display.clear_output(wait=True)
with open(f'{gds}.gltf') as f:
  gltf_data=f.read()
  output = gds_viewer.render(gltf_data=gltf_data)
IPython.display.HTML(output)

In [ ]:
#@title メトリック {display-mode: "form"}
#@markdown [ドキュメント](https://openlane.readthedocs.io/en/latest/reference/datapoint_definitions.html)
import pathlib
import pandas as pd

pd.options.display.max_rows = None
csv = sorted(pathlib.Path('runs').glob('*/reports/metrics.csv'))
df = pd.read_csv(csv[-1])
df.transpose()

# 演習課題

##乗算器

### ⌨ DSLX

1. 以下の関数 `mul4` を変更して4ビットの乗算器を実装してください。実装にあたっては、[DSLXのスタンダードライブラリ](https://google.github.io/xls/dslx_std/) の関数を使ってみましょう（`std::`のプレフィックスを忘れずに）。
1. この乗算器のVerilogコードを生成してください。
1. OpenLaneのフローを走らせて回路合成まで進めてください。
1. グラフの複雑さを加算器と比較してみてください。

In [ ]:
%%bash -c 'cat > user_module.x; interpreter_main user_module.x'
import std

fn mul4(a: u4, b: u4) -> u8 {
  u8:0 // TODO(YOU) implement mul4
}

fn user_module(io_in: u8) -> u8 {
  mul4(io_in[0:4], io_in[4:8]) as u8
}

#[test]
fn test() {
  let _ = assert_eq(mul4(u4:8, u4:8), u8:64);
  let _ = assert_eq(user_module(u8:0b1000_1000), u8:0b0100_0000);
  _
}

In [ ]:
!ir_converter_main --top=user_module user_module.x > user_module.ir
!opt_main user_module.ir > user_module_opt.ir
!codegen_main --use_system_verilog=false --module_name=user_module --generator=combinational user_module_opt.ir > user_module.v
!cat user_module.v

In [ ]:
%env PDK=sky130A
!flow.tcl -design . -to synthesis

In [ ]:
#@title プレビュー {display-mode: "form"}

import graphviz
import pathlib

dots = sorted(pathlib.Path('runs').glob('*/tmp/synthesis/post_techmap.dot'))
dot = graphviz.Source.from_file(dots[-1])
dot.engine = 'dot'
dot

### 🛠️ OpenLane

1. OpenLaneのフローを最後まで走らせてください。
1. `DIE_AREA`または`PL_TARGET_DENSITY` の[設定値](https://openlane.readthedocs.io/en/latest/reference/configuration.html)を変更することでエラーが発生しないように調整してください。
1. [メトリック](https://openlane.readthedocs.io/en/latest/reference/datapoint_definitions.html) (`Total_Physical_Cells`, `wire_length`)を加算器の場合と比較してください。
1. レイアウトを加算器の場合と比較してください。

In [ ]:
%%writefile config.json
{
    "DESIGN_NAME": "user_module",
    "VERILOG_FILES": "dir::user_module.v",
    "CLOCK_TREE_SYNTH": false,
    "CLOCK_PERIOD": 10,
    "CLOCK_PORT": "clk",
    "CLOCK_NET": "ref::$CLOCK_PORT",
    "FP_SIZING": "absolute",
    "DIE_AREA": "0 0 50 50",
    "PL_TARGET_DENSITY": 0.30,
    "FP_PIN_ORDER_CFG": "dir::pin_order.cfg"
}

In [ ]:
%env PDK=sky130A
!flow.tcl -design .

In [ ]:
#@title メトリック {display-mode: "form"}
import pathlib
import pandas as pd

pd.options.display.max_rows = None
csv = sorted(pathlib.Path('runs').glob('*/reports/metrics.csv'))
df = pd.read_csv(csv[-1])
df.transpose()

In [ ]:
#@title プレビュー {display-mode: "form"}

import pathlib

gds = sorted(pathlib.Path('runs').glob(f'*/results/final/gds/*.gds'))[-1].resolve()
!python3 GDS2glTF/gds2gltf.py {gds} | tee gds2gltf.log
import IPython.display
IPython.display.clear_output(wait=True)
with open(f'{gds}.gltf') as f:
  gltf_data=f.read()
  output = gds_viewer.render(gltf_data=gltf_data)
IPython.display.HTML(output)

##パイプライン

### パイプラインのステージ
- 乗算器を複数のパイプラインステージに[スケジュール](https://google.github.io/xls/scheduling/)するために、XLSの[codegen pipeliningの設定](https://google.github.io/xls/codegen_options/#pipelining-and-scheduling-options)の適切な組み合わせを探してください。
- 生成されたverilogコードを確認してください。


In [ ]:
pipeline_stages = 1
clock_period_ps = 10
!ir_converter_main --top=user_module user_module.x > user_module.ir
!opt_main user_module.ir > user_module_opt.ir
!codegen_main --pipeline_stages={pipeline_stages} --clock_period_ps={clock_period_ps} --delay_model=sky130 --use_system_verilog=false --module_name=user_module user_module_opt.ir > user_module.v
!cat user_module.v

### シミュレーション
- [cocotb](https://www.cocotb.org/)テストベンチをアップデートして、生成されたverikogの動作を確認してください。
- [タイミング図]((https://en.wikipedia.org/wiki/Digital_timing_diagram))を確認して値がどのようにパイプラインのステージを流れていくかを確認してください。

In [ ]:
%%writefile Makefile
SIM ?= icarus
TOPLEVEL_LANG ?= verilog
VERILOG_SOURCES += user_module.v
VERILOG_SOURCES += top.v
TOPLEVEL = top
MODULE = test

include $(shell cocotb-config --makefiles)/Makefile.sim

In [ ]:
%%writefile top.v

module top(
  input wire clk,
  input wire [7:0] io_in,
  output wire [7:0] out
);

  user_module user_module_tb(clk, io_in, out);
`ifdef COCOTB_SIM
initial begin
  $dumpfile ("user_module.vcd");
  $dumpvars (0, user_module_tb);
  #1;
end
`endif
endmodule

In [ ]:
%%bash -c "cat > test.py; make"
import cocotb
from cocotb.triggers import ClockCycles, RisingEdge
from cocotb.clock import Clock

@cocotb.test()
async def test_test(dut):
    c = Clock(dut.clk, 1, 'ns')
    cocotb.start_soon(c.start())

    # TODO(YOU): update with the 8-input to user_module
    dut.user_module_tb.io_in.value = NaN
    # pipeline stage 0: fetch
    await ClockCycles(dut.clk, 2)
    # TODO(YOU): verify that the input value propagate to stage #0
    assert dut.user_module_tb.p0_io_in == NaN

    # pipeline stage 1: slice
    await ClockCycles(dut.clk, 2)
    # TODO(YOU): verify that the input value is sliced into 4-bit umul operands
    assert dut.user_module_tb.p1_bit_slice_TODO == NaN
    assert dut.user_module_tb.p1_bit_slice_TODO == NaN

    # pipeline stage 2: multiply
    await ClockCycles(dut.clk, 2)
    # TODO(YOU): verify that the output value match the multiplication result
    assert dut.user_module_tb.out == NaN

In [ ]:
#@title プレビュー {display-mode: "form"}

import wavedrom
import vcdvcd
import json

def filter_signals(top, signals, filter=[]):
  if len(filter) == 0:
    return signals
  for f in filter:
    for s in signals:
      if f in s and 'comb' not in s:
        yield s.replace(f'{top}.', '')

def signals_to_wave(filename, top, clk='clk', cycles=0, filter=[], period=500):
  vcd = vcdvcd.VCDVCD(filename)
  clk_tv = vcd[f'{top}.{clk}'].tv
  ticks = len(clk_tv) if cycles == 0 else cycles * 2
  yield {'name': clk, 'wave': 'P' + '.' * (cycles-1), 'period': 2 }
  for s in filter_signals(top, vcd.signals, filter):
    wave = ['x'] * ticks
    data = []
    for t, d in vcd[f'{top}.{s}'].tv:
      i = int(t/500)
      if d.isnumeric():
        wave[i] = '='
        data.append(hex(int(f'0b{d}', 2)))
      else:
        wave[i] = d
    yield {'name': s, 'wave': ''.join(wave), 'data': ' '.join(data), 'phase': 0.5}

drom = {
    'signal': list(signals_to_wave('user_module.vcd', top='top.user_module_tb', cycles=pipeline_stages+1, filter=['io_in', 'p1_bit_slice', 'out']))
}
svg = wavedrom.render(json.dumps(drom))
display(svg)

### 静的タイミング解析
- [静的タイミング解析](https://xtech.nikkei.com/dm/article/WORD/20090107/163760/)を実行してください。
- ログにある全ての[コーナ](https://en.wikipedia.org/wiki/Static_timing_analysis#Corners_and_STA)に関して、[スラック](https://xtech.nikkei.com/dm/article/WORD/20090107/163760/#:~:text=%E3%81%AE%E5%88%A4%E5%AE%9A%E3%82%92%E3%80%8C-,%E3%82%B9%E3%83%A9%E3%83%83%E3%82%AF,-%E3%80%8D%E3%81%A8%E3%81%84%E3%81%86%E9%87%8F%E3%81%A7)が`VIOLATED`から`MET`に変わるまで、`CLOCK_PERIOD`を変更してください。
- ハードウェア合成を再度行ってください。
- レイアウト上のパイプラインにどのような影響があるかを確認してください。

In [ ]:
%%writefile config.json
{
    "DESIGN_NAME": "user_module",
    "VERILOG_FILES": "dir::user_module.v",
    "CLOCK_TREE_SYNTH": true,
    "CLOCK_PERIOD": 1,
    "CLOCK_PORT": "clk",
    "CLOCK_NET": "ref::$CLOCK_PORT",
    "FP_SIZING": "absolute",
    "DIE_AREA": "0 0 150 150",
    "PL_TARGET_DENSITY": 0.70,
    "FP_PIN_ORDER_CFG": "dir::pin_order.cfg"
}

In [ ]:
%%writefile pin_order.cfg
#BUS_SORT

#W
io_in\[0\]
io_in\[1\]
io_in\[2\]
io_in\[3\]
clk
io_in\[4\]
io_in\[5\]
io_in\[6\]
io_in\[7\]

#E
out.*

In [ ]:
%env PDK=sky130A
!flow.tcl -design . -verbose 10 -to cts

In [ ]:
%env PDK=sky130A
!flow.tcl -design .

In [ ]:
#@title メトリック {display-mode: "form"}
import pathlib
import pandas as pd

pd.options.display.max_rows = None
csv = sorted(pathlib.Path('runs').glob('*/reports/metrics.csv'))
df = pd.read_csv(csv[-1])
df.transpose()

In [ ]:
#@title プレビュー {display-mode: "form"}

import pathlib

gds = sorted(pathlib.Path('runs').glob(f'*/results/final/gds/*.gds'))[-1].resolve()
!python3 GDS2glTF/gds2gltf.py {gds} | tee gds2gltf.log
import IPython.display
IPython.display.clear_output(wait=True)
with open(f'{gds}.gltf') as f:
  gltf_data=f.read()
  output = gds_viewer.render(gltf_data=gltf_data)
IPython.display.HTML(output)

## チャレンジ課題

1. 残りの時間を使って、自由にチップ設計に挑戦してみてください。
1. XLSの[stdlib](https://github.com/google/xls/tree/main/xls/dslx/stdlib)、[examples](https://github.com/google/xls/tree/main/xls/examples)、[modules](https://github.com/google/xls/tree/main/xls/examples)を使ったり[third_party](https://github.com/google/xls/tree/main/third_party)のものを再利用してみてもよいでしょう。

In [ ]:
%%bash -c 'cat > user_module.x; interpreter_main user_module.x'

fn user_module(io_in: u8) -> u8 {
  u8:0
}

#[test]
fn test() {
  let _ = assert_eq(user_module(u8:0), u8:0);
  _
}

In [ ]:
!ir_converter_main --top=user_module user_module.x > user_module.ir
!opt_main user_module.ir > user_module_opt.ir
!codegen_main --use_system_verilog=false --module_name=user_module --generator=combinational user_module_opt.ir > user_module.v
!cat user_module.v

In [ ]:
%env PDK=sky130A
!flow.tcl -design .

In [ ]:
#@title Metrics {display-mode: "form"}
import pathlib
import pandas as pd

pd.options.display.max_rows = None
csv = sorted(pathlib.Path('runs').glob('*/reports/metrics.csv'))
df = pd.read_csv(csv[-1])
df.transpose()

In [ ]:
#@title Preview {display-mode: "form"}

import pathlib

gds = sorted(pathlib.Path('runs').glob(f'*/results/final/gds/*.gds'))[-1].resolve()
!python3 GDS2glTF/gds2gltf.py {gds} | tee gds2gltf.log
import IPython.display
IPython.display.clear_output(wait=True)
with open(f'{gds}.gltf') as f:
  gltf_data=f.read()
  output = gds_viewer.render(gltf_data=gltf_data)
IPython.display.HTML(output)

# テープアウト
Googleがスポンサーする[OpenMPW シャトルプログラム](https://developers.google.com/silicon)に参加するためには、[Caravel ユーザープロジェクト](https://caravel-user-project.readthedocs.io/en/latest/) を使って、作成したデザインを[Caravel SoC ハーネス](https://caravel-harness.readthedocs.io/)と統合し、[mpw pre-check](https://github.com/efabless/mpw_precheck)のテストを通す必要があります。


![caravel](https://caravel-user-project.readthedocs.io/en/latest/_static/layout.png)

- ユーザープロジェクトのサイズ: 2920 µm x 3520 µm
- SoC: RISC-Vベース ([スペック](https://caravel-mgmt-soc-litex.readthedocs.io/en/latest/#features))
- I/Oピン: 38 ([ピン配列](https://caravel-harness.readthedocs.io/en/latest/pinout.html))
- 通信: Wishboneバスおよび128ピン ロジックアナライザー

## `tiny_user_project`を使う方法

1. [tiny_user_projectのテンプレート](https://github.com/proppy/tiny_user_project)を使って新しいGitHubプロジェクトを作成する。
1. [GitHubのページで`Sources`を`GitHub Actions`に設定する。](https://tinytapeout.com/faq/#my-github-action-is-failing-on-the-pages-part)
1. `verilog/rtl/user_module.v`を自分のデザイン用のverilogコードに置き換える。
1. `info.yaml`を自分のプロジェクト用に書き換える:
   - `wokwi_id`を`0`にセットする。
   - `source_files`のコメントを外し`- verilog/rtl/user_module.v`に設定する。
   - `top_module`のコメントを外し、`user_module`に設定する。
   - `inputs` / `outputs`のマッピングが自分のプロジェクトにあうように`documentation` をアップデートする。
1. 変更をコミット・プッシュして ![user_project_ci](https://github.com/proppy/tiny_caravel_user_project/actions/workflows/user_project_ci.yml/badge.svg)ワークフローの`Actions`のサマリーを確認する（成功していれば、自動的に生成されたファイルを含む新規コミットが作られる）。

1. プロジェクトのGitHubレポジトリを次回の[Open MPW シャトル](https://platform.efabless.com/shuttles/MPW-8)に[送信](https://platform.efabless.com/projects/create?project_definition=Open+MPW&shuttle=MPW-8)する。